# Week 6 Practice Exercises

These practice exercises demonstrate how to i) use <a href="https://planetarycomputer.microsoft.com/catalog" target="_blank">Microsoft's Planetary Computer</a> to generate a monthly data cube (rank 3 NumPy `ndarray`) of NDVI values from the Landsat satellite sensor, and ii) use zonal statistics operations to generate a monthly NDVI time-series for a collection of tree crop plantation boundaries in Western Australia. 

## Setup

### Run the labs

You can run the labs locally on your machine or you can use cloud environments provided by Google Colab. **If you're working with Google Colab be aware that your sessions are temporary and you'll need to take care to save, backup, and download your work.**

<a href="https://colab.research.google.com/github/data-analysis-3300-3003/colab/blob/main/lab-6-practice-exercises.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Download data

If you need to download the date for this lab, run the following code snippet. 

In [ ]:
import os

if "week-6-practice" not in os.listdir(os.getcwd()):
    os.system('wget "https://github.com/data-analysis-3300-3003/data/raw/main/data/week-6-practice.zip"')
    os.system('unzip "week-6-practice.zip"')

### Import modules

In [ ]:
import os
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import pystac_client
import planetary_computer as pc
import plotly.express as px
import plotly.io as pio
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
from skimage import io


from pystac.extensions.eo import EOExtension as eo

# setup renderer
if 'google.colab' in str(get_ipython()):
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "jupyterlab"

In [ ]:
!pip install rasterstats

In [ ]:
from rasterstats import zonal_stats

### Load data

Load a small vector dataset of tree crop plantations near Carnarvon, Western Australia, derived from the <a href="https://www.agriculture.gov.au/abares/aclump/land-use/catchment-scale-land-use-of-australia-commodities-update-december-2020" target="_blank">Catchment scale land use of Australia (CLUM) dataset</a>.

In [ ]:
clum_demo_path = os.path.join(os.getcwd(), "week-6-practice", "clum_demo_aoi.gpkg")
clum_aoi = gpd.read_file(clum_demo_path)

In [ ]:
clum_aoi.head()

In [ ]:
clum_aoi.explore(column="Commod_dsc")

### Bounding box

To generate a bounding box around the polygon geometries that comprise the `GeoDataFrame` `clum_aoi` we need to perform three operations:

1. combine all the geometry objects on separate rows of `clum_aoi` into one multipolygon object using the `unary_union` property of the `geometry` `GeoSeries` in `clum_aoi`.
2. convert the result of the `unary_union` operation to a `GeoSeries` object.
3. get the `envelope` of the `GeoSeries` of the `unary_union` result. The envelope is the smallest rectangular bounding box that surrounds a geometry.  

If we did not perform the `unary_union` operation to combine all the polygon geometries in `clum_aoi` into a single multipolygon object, we'd end up computing the envelope for each of the separate polygon objects that comprise the `GeoDataFrame`. This would not return to us a bounding box surrounding all the polygons in our `GeoDataFrame`.

In [ ]:
clum_bbox = gpd.GeoSeries(clum_aoi.geometry.unary_union).envelope

### Coordinate Reference Systems

We can inspect the coordinate reference system (CRS) of the CLUM geometry data that we read into our program by printing its `crs` property. We can see the CLUM data has a CRS of `epsg:4283` in `AUTHORITY:CODE` format - `epsg` stands for European Petroleum Survey Group which publishes a list of CRS and `4283` is the CRS code. You can look up the CRS here: https://spatialreference.org/ and find a more detailed explanation of CRS in the <a href="https://r.geocompx.org/reproj-geo-data.html#crs-in-r" target="_blank">Geocomputation with R book's section on Coordinate Reference Systems</a>.

In [ ]:
print("The CRS of the clum_gpd GeoDataFrame is:")
print(clum_aoi.crs)

In [ ]:
clum_aoi.crs

The bounding box `GeoSeries` that we generated has lost its `crs` property. We can use the `set_crs()` method to pass in a CRS code and reset the CRS for the spatial data. 

In [ ]:
clum_bbox = clum_bbox.set_crs("EPSG:4283")

If we want to transform the CRS of a `GeoSeries` or `GeoDataFrame` object, we can use the `to_crs()` method. `EPSG:4326` is a commonly used format for latitude and longitude data.

In [ ]:
clum_bbox = clum_bbox.to_crs("EPSG:4326")

#### Task

**Head to the GeoPandas docs and make sure you're familiar with the `unary_union`, `envelope`, `set_crs()`, and `to_crs()` properties and operations.**

### Generate monthly NDVI rasters 

Now we have a bounding box geometry for our `GeoDataFrame` of polygons denoting tree crop plantations, we can use this bounding box to download images data from satellites intersecting this location for each month of the year and compute an NDVI raster for each month.

First, we need to create a `pystac_client.Client` object for the Microsoft Planetary Computer STAC API.

In [ ]:
# open a connection to the Microsoft Planetary Computer's root STAC catalog
pc_catalog = pystac_client.Client.open(
    url="https://planetarycomputer.microsoft.com/api/stac/v1",
    # modifier=planetary_computer.sign_inplace
)

Next, we'll create a small routine that will loop over a sequence of months (numbers 1 to 12). 

For each month, we'll use the `pc_catalog` object's `search()` method to query the Microsoft Planetary Computer STAC API for all Landsat <a href="https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2" target="_blank">Landsat Collection 2 Level 2</a> images that intersect our bounding box.

You will notice we pass a query into the `search()` method: 

```
query={
    "platform": {"in": ["landsat-8"]},
}
```

This query is restricting our search to images from the Landsat 8 sensor; the Landsat 7 sensor which was also operational during this period was affected by a scan line corrector failure resulting in data gaps. To include both Landsat 7 and 8 sensors you could remove this query. You can find more information about Landsat <a href="https://landsat.gsfc.nasa.gov/" target="_blank">here</a>.

Once the search has returned to us an `ItemCollection` of Landsat image assets that matched our search criteria, we'll find the least cloudy Landsat image and download the red and near infrared bands. This data is stored in cloud optimised GeoTIFF (COG) files; see the below notes on how to read data from COG files for a region of interest (`Window`). 

Once we have downloaded the red and near infrared data for our region of interest we cast this to `float32` `dtype` and compute the NDVI.  

#### Download COG data

We can read data from COG files in the cloud using rasterio in a similar way to how we've been reading local GeoTIFF files on our machine. 

We use the `rasterio.open()` function to open a file connection to the COG in the cloud and use the connection object's `read()` method to read data from the COG in the cloud to a NumPy `ndarray` on our machine. 

However, to read in a subset of the data we use the `window` argument of `read()` and pass in a `Window` object. 

A `Window` object is a rectangular subset of raster defined as `Window(column_offset, row_offset, width, height)`. 

The rasterio.windows module has a <a href="https://rasterio.readthedocs.io/en/latest/api/rasterio.windows.html#rasterio.windows.from_bounds" target="_blank">`from_bounds()`</a> function which converts bounding coordinates to a `Window` object. 

rasterio has a `features` module which has as a `bounds()` function which takes in a GeoJSON geometry or Shapely `geometry` and returns a (left, bottom, right, top) bounding box which we can use to create a `Window`. 

However, our GeoJSON geometry will be in EPSG:4326 (geographic) coordinate system which could be different from the CRS of the COG we're trying to read data from. This requires us to use rasterio `transform_bounds()` function to transform our bounding box coordinates to the coordinates of the COG data. 

To summarise this process: 

1. use `features.bounds()` to convert GeoJSON or Shapely `geometry` to a bounding box.
2. use `warp.transform_bounds()` to convert the bounding box to the CRS of the COG data.
3. use `windows.from_bounds()` to convert the reprojected bounding box to a `Window` object.
4. pass the `Window` object to `read()` to read only data from the COG within the `Window`. 

#### Working with collections

We are reading Landsat data from images that form a STAC Item Collection - the <a href="https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2" target="_blank">Landsat Collection 2 Level 2</a> collection. The images in the collection can come from different sensors, have different coordinate reference systems, and overlap. Thus, to ensure that the data we read from different images to match our `Window` has the same shape, we can pass a shape into the `out_shape` argument of rasterio's `read()` method. 

Here, we get the shape of the first NumPy `ndarray` that we read from a COG file. Then, for all other `read()` operations we pass this shape into the `out_shape` argument. You can see this implemented below in the `if` and `else` blocks.  

In [ ]:
months = range(1, 13)
year = 2016
year_next = 2016

# get the bounding box as a shapely object
clum_bbox_shapely = clum_bbox[0]

# empty list to store ndarray of monthly ndvi values
ndvi_arr = []

for i in months:
    print(f"processing month {i}")
    
    # here we convert numeric month indicators which can be single digit (e.g. Jan = 1) 
    # to string month indicators which are two digits (e.g. Jan = "01")
    if i < 10:
        mnth = str(0) + str(i)
    else:
        mnth = str(i)
    
    # create month and year indicators for the start of the next month
    # we do this to constrain the search to one month at a time
    if i < 9:
        mnth_next = str(0) + str(i + 1)
    else:
        mnth_next = str(i + 1)
        
    if mnth_next == "13":
        mnth_next = "01"
        year_next = year + 1
    
    # create a time of interest search for each month in turn
    time_of_interest = f"{year}-{mnth}-01/{year_next}-{mnth_next}-01"
    
    # search the Microsoft Planetary computer to find Landsat assets matching our bounding box and time of interest
    search = pc_catalog.search(
        collections=["landsat-c2-l2"],
        intersects=clum_bbox_shapely, # use shapely object
        datetime=time_of_interest,
        query={
            "platform": {"in": ["landsat-8"]},
        }
    )

    # items is an item collection of Landsat assets that matched the search criteria
    items = search.item_collection()
    
    # get item with lowest cloud cover per-month
    
    # empty list - append the cloud cover each Landsat asset in the item collection to the list
    cloud_cover = []
    for cld in items:
        cloud_cover.append(eo.ext(cld).cloud_cover)
        
    # find the index location in the list of the asset with lowest cloud cover
    min_cloud_cover = min(cloud_cover)
    min_cloud_cover_idx = cloud_cover.index(min_cloud_cover)
    
    # get the Landsat asset with the lowest cloud cover
    least_cloudy = items[min_cloud_cover_idx]
    
    # get the signed URL for the red and nir least cloudy Landsat assets
    least_cloudy_red_href = pc.sign(least_cloudy.assets["red"].href)
    least_cloudy_nir08_href = pc.sign(least_cloudy.assets["nir08"].href)
    
    # for the first month get the shape of the array
    # for subequent months resample all arrays to this shape
    if i == 1:
        # read red band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_red_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            red = src.read(1, window=aoi_window)
            
            # get affine transform for the windowed read
            # this allows us to reproject the windowed read of data from COG file
            src_transform = src.transform
            win_transform = src.window_transform(aoi_window)
            
            # create a copy of the metadata for saving later
            out_meta = src.meta
            
            # get the shape of the red ndarray
            # force all other rasters to match this shape
            out_shape = red.shape
            
        # read nir band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_nir08_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            nir = src.read(1, out_shape=out_shape, window=aoi_window)
    
        # compute NDVI
        # cast to float as NDVI is bound between -1 and 1 
        nir = nir.astype("float32")
        red = red.astype("float32")
        ndvi = (nir - red) / (nir + red)

    else:
        # read red band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_red_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            # Here we pass out_shape into read to define the shape of the array data is read into
            red = src.read(1, out_shape=out_shape, window=aoi_window)
            
        # read nir band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_nir08_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            # Here we pass out_shape into read to define the shape of the array data is read into
            nir = src.read(1, out_shape=out_shape, window=aoi_window)
    
        # compute NDVI
        # cast to float as NDVI is bound between -1 and 1 
        nir = nir.astype("float32")
        red = red.astype("float32")
        ndvi = (nir - red) / (nir + red)
        
    # append NDVI ndarray to list
    ndvi_arr.append(ndvi)

# stack the ndvi ndarray's to create a multiband raster
ndvi_stacked = np.stack(ndvi_arr, axis=0)
print(f"the shape of ndvi_arr is {ndvi_stacked.shape}")

We can inspect the metadata of the COG file we are reading data from. We can see that its width and height are far greater than width and height of the windowed read of data that corresponded to our bounding box (width = 59, and height = 46). This is because the metadata corresponds to the entire Landsat scene, but we extracted a small subset of this data that intersects with our bounding box. This is why we computed the `win_transform` object above as this affine transform object allows us to reproject the `ndarray` generated by the windowed read (i.e. relate elements in the NumPy `ndarray` to positions on the Earth's surface). You can read about affine transforms in GIS <a href="https://pygis.io/docs/d_affine.html" target="_blank">here</a>.

In [ ]:
# inspect the metadata
out_meta

To save our monthly multiband representation of NDVI rasters, we need to create new metadata that describes the data. We can do this by updating the various fields in the metadata dictionary object. We can then save the multiband raster to file using the standard Python / rasterio approach of opening a file connection object in write mode. 

In [ ]:
# update meta for saving
out_meta["dtype"] = "float32"
out_meta["count"] = 12
out_meta["height"] = ndvi_arr[0].shape[0]
out_meta["width"] = ndvi_arr[0].shape[1]
out_meta["transform"] = win_transform
out_meta

In [ ]:
# save multiband NDVI raster
ndvi_out_path = os.path.join(os.getcwd(), "week-6-practice", "ndvi_by_month.tif")

with rasterio.open(ndvi_out_path, "w", **out_meta) as dst:
        dst.write(ndvi_stacked, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

### Zonal statistics

We can use the <a href="https://pythonhosted.org/rasterstats/manual.html#raster-data-sources" target="_blank">rasterstats package zonal_stats()</a> function to compute zonal statistics that summarise the NDVI values within each of the plantation boundary geometries. 

To do this, we can loop over the list of `ndarray`s which store monthly NDVI values and compute the mean NDVI for all pixels that intersect with a plantation geometry. For each call to the `zonal_stats()` function we need to pass in the `GeoDataFrame` of plantation boundaries, `clum_aoi` transformed to the CRS of the NDVI data. We also need to pass in the NumPy `ndarray` of NDVI values and the affine transform that relates `ndarray` elements to locations on the Earth's surface. 

For each month, we can create a new column in the `GeoDataFrame` `clum_aoi_ndvi` and append the results of the zonal statistics operation. 

In [ ]:
# zonal stats
zstats_meta = out_meta
zstats_meta["count"] = 1
# create a copy of clum_aoi to save zonal_stats results to
clum_aoi_ndvi = clum_aoi.copy()

month_idx = 1
for i in ndvi_arr:  
    print(f"computing ndvi zonal stats for month {month_idx}")
    
    zstats = zonal_stats(
        clum_aoi.to_crs(zstats_meta["crs"]), 
        i, 
        affine=zstats_meta["transform"], 
        stats=["mean"], 
        all_touched=True
    )
    
    df_zstats = pd.DataFrame(zstats)
    clum_aoi_ndvi[f"ndvi_{month_idx}"] = df_zstats.iloc[:, 0] 
    month_idx += 1

In [ ]:
clum_aoi_ndvi.head()

In [ ]:
# visualise average monthly NDVI for row 4
px.line(
    x=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    y=clum_aoi_ndvi.loc[3, "ndvi_1":"ndvi_12"].tolist(),
    labels = {
        "y": "NDVI",
        "x": "month"
    }   
)

#### Recap quiz

**Can you repeat the above process of generating monthly NDVI arrays and computing zonal statistics for plantation boundaries using data from the Sentinel-2 sensor?**

**Things to note: the name of the Sentinel-2 STAC Item Collection and the band names for red and near infrared reflectance.**

In [ ]:
## ADD CODE HERE ##

<details>
    <summary><b>answer - download S2 assets</b></summary>
    
```python
months = range(1, 13)
year = 2016
year_next = 2016

# get the bounding box as a shapely object
clum_bbox_shapely = clum_bbox[0]

# empty list to store ndarray of monthly ndvi values
s2_ndvi_arr = []

for i in months:
    print(f"processing month {i}")
    
    # here we convert numeric month indicators which can be single digit (e.g. Jan = 1) 
    # to string month indicators which are two digits (e.g. Jan = "01")
    if i < 10:
        mnth = str(0) + str(i)
    else:
        mnth = str(i)
    
    # create month and year indicators for the start of the next month
    # we do this to constrain the search to one month at a time
    if i < 9:
        mnth_next = str(0) + str(i + 1)
    else:
        mnth_next = str(i + 1)
        
    if mnth_next == "13":
        mnth_next = "01"
        year_next = year + 1
    
    # create a time of interest search for each month in turn
    time_of_interest = f"{year}-{mnth}-01/{year_next}-{mnth_next}-01"
    
    # search the Microsoft Planetary computer to find Sentinel-2 assets matching our bounding box and time of interest
    search = pc_catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=clum_bbox_shapely, # use shapely object
        datetime=time_of_interest
    )

    # items is an item collection of S2 assets that matched the search criteria
    items = search.item_collection()
    
    # get item with lowest cloud cover per-month
    
    # empty list - append the cloud cover each S2 asset in the item collection to the list
    cloud_cover = []
    for cld in items:
        cloud_cover.append(eo.ext(cld).cloud_cover)
        
    # find the index location in the list of the asset with lowest cloud cover
    min_cloud_cover = min(cloud_cover)
    min_cloud_cover_idx = cloud_cover.index(min_cloud_cover)
    
    # get the S2 asset with the lowest cloud cover
    least_cloudy = items[min_cloud_cover_idx]
    
    # get the signed URL for the red and nir least cloudy S2 assets
    least_cloudy_red_href = pc.sign(least_cloudy.assets["B04"].href)
    least_cloudy_nir08_href = pc.sign(least_cloudy.assets["B08"].href)
    
    # for the first month get the shape of the array
    # for subequent months resample all arrays to this shape
    if i == 1:
        # read red band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_red_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            red = src.read(1, window=aoi_window)
            
            # get affine transform for the windowed read
            # this allows us to reproject the windowed read of data from COG file
            src_transform = src.transform
            win_transform = src.window_transform(aoi_window)
            
            # create a copy of the metadata for saving later
            out_meta = src.meta
            
            # get the shape of the red ndarray
            # force all other rasters to match this shape
            out_shape = red.shape
            
        # read nir band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_nir08_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            nir = src.read(1, out_shape=out_shape, window=aoi_window)
    
        # compute NDVI
        # cast to float as NDVI is bound between -1 and 1 
        nir = nir.astype("float32")
        red = red.astype("float32")
        ndvi = (nir - red) / (nir + red)

    else:
        # read red band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_red_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            # Here we pass out_shape into read to define the shape of the array data is read into
            red = src.read(1, out_shape=out_shape, window=aoi_window)
            
        # read nir band
        # open a connection to the COG using its signed link
        with rasterio.open(least_cloudy_nir08_href) as src:
            aoi_bounds = features.bounds(clum_bbox_shapely)
            warped_aoi_bounds = warp.transform_bounds("epsg:4326", src.crs, *aoi_bounds)
            aoi_window = windows.from_bounds(transform=src.transform, *warped_aoi_bounds)
            # Here we pass out_shape into read to define the shape of the array data is read into
            nir = src.read(1, out_shape=out_shape, window=aoi_window)
    
        # compute NDVI
        # cast to float as NDVI is bound between -1 and 1 
        nir = nir.astype("float32")
        red = red.astype("float32")
        ndvi = (nir - red) / (nir + red)
        
    # append NDVI ndarray to list
    s2_ndvi_arr.append(ndvi)

# stack the ndvi ndarray's to create a multiband raster
s2_ndvi_stacked = np.stack(s2_ndvi_arr, axis=0)
print(f"the shape of ndvi_arr is {s2_ndvi_stacked.shape}")

```
</details>

<details>
    <summary><b>answer - S2 zonal stats</b></summary>
    
```python
# update meta for saving
out_meta["dtype"] = "float32"
out_meta["count"] = 12
out_meta["height"] = s2_ndvi_arr[0].shape[0] ## NOTE - using S2 output here
out_meta["width"] = s2_ndvi_arr[0].shape[1]
out_meta["transform"] = win_transform
out_meta

# zonal stats
zstats_meta = out_meta
zstats_meta["count"] = 1
# create a copy of clum_aoi to save zonal_stats results to
s2_clum_aoi_ndvi = clum_aoi.copy()

month_idx = 1
## NOTE - loop over S2 ndvi arrays
for i in s2_ndvi_arr:  
    print(f"computing ndvi zonal stats for month {month_idx}")
    
    zstats = zonal_stats(
        clum_aoi.to_crs(zstats_meta["crs"]), 
        i, 
        affine=zstats_meta["transform"], 
        stats=["mean"], 
        all_touched=True
    )
    
    df_zstats = pd.DataFrame(zstats)
    s2_clum_aoi_ndvi[f"ndvi_{month_idx}"] = df_zstats.iloc[:, 0] 
    month_idx += 1
    
s2_clum_aoi_ndvi.head()
```
</details>

<p></p>

If you inspect the above NDVI time-series generated from Sentinel-2 data, you can spot that for some months there is missing data. We might want to explore why this is the case. STAC Items often have *preview* or *thumbnail* assets that we can use to quickly inspect the data. Let's look into why there is missing data for the month of May. 

First, let's create a search of the Sentinel-2 data for May. This will return to us an Item Collection object. 

In [ ]:
# create a time of interest search for each month in turn
time_of_interest = f"2016-05-01/2016-06-01"

# search the Microsoft Planetary computer to find Sentinel-2 assets matching our bounding box and time of interest
search = pc_catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=clum_bbox_shapely, # use shapely object
    datetime=time_of_interest
)

# items is an item collection of S2 assets that matched the search criteria
items = search.item_collection()

In [ ]:
items

Now, let's see what assets are available for each Item in the Item Collection. 

In [ ]:
items[0].assets

We can see there is a `rendered_preview` asset that's a PNG file. We can load that file into our program and quickly visualise it. To look at other Item's preview change the index location into `items`.  

In [ ]:
img = io.imread(pc.sign(items[5].assets["rendered_preview"].href))
px.imshow(img, height=600)